機械学習のアルゴリズムを用いるより先に、データの特性を理解して前処理を行う事がかなり重要  
データサイエンティストの仕事の8～9割はデータ前処理であるとも言われている  
  
主な作業としては  
* 欠損値への対応
* カテゴリ変数のエンコード
* 特徴量の正規化
  
データの前処理を行う方法として、pandasをある程度使ったが、ここからはscikit-learnを用いる  
  
scikit-learnを用いるメリットは  
* fit , transform メソッドが使える
* scikit-learn全体で統一的なインターフェースである
  
というところ  
  
scikit-learnはpythonで機械学習を行う際のデファクトスタンダードとなっている

### 欠損値への対応
欠損値は、測定器の不具合、通信機の不具合で発生することが多い。  
実際の業務の中で欠損値は必ず遭遇し、その量は膨大。  
欠損値があると、今後の学習や統計処理に悪影響を及ぼすため対応が必要  
  
主な対応方法としては次の通り  
* 欠損値を除去する
* 欠損値を補完する


In [1]:
import numpy
import pandas

# サンプルデータセットの作成
data_frame = pandas.DataFrame({"A":[1,numpy.nan,3,4,5],
                               "B":[6,7,8,numpy.nan,10],
                               "C":[11,12,13,14,15],})

data_frame

,A,B,C
0,1.0,6.0,11
1,NaN,7.0,12
2,3.0,8.0,13
3,4.0,NaN,14
4,5.0,10.0,15


### 欠損値の除去

In [2]:
# 要素に欠損値があるかを調べる
data_frame.isnull()

,A,B,C
0,False,False,False
1,True,False,False
2,False,False,False
3,False,True,False
4,False,False,False


isnullを用いると、欠損値がどこにあるのかが分かる  
もしこれで欠損値を除去したいのであれば、次のようにできる

In [4]:
drop_nan = data_frame.dropna()
drop_nan

,A,B,C
0,1.0,6.0,11
2,3.0,8.0,13
4,5.0,10.0,15


### 欠損値の補完  
欠損値を他の妥当な値で補うという方法もある  
pandasではfillnaメソッドでこれを行うことが出来る  
もう一つ、scikit-learnのpreprocessingを用いる方法もある

In [7]:
from sklearn.impute import SimpleImputer

# strategyにmeanを指定することで、平均値による補完を行えるインスタンスを作成する
imputer = SimpleImputer(strategy="mean")

# 欠損値の補完
imputer.fit(data_frame)

# 返り値はNumpyの配列になっているところに注意する
imputer.transform(data_frame)

array([[ 1.  ,  6.  , 11.  ],
       [ 3.25,  7.  , 12.  ],
       [ 3.  ,  8.  , 13.  ],
       [ 4.  ,  7.75, 14.  ],
       [ 5.  , 10.  , 15.  ]])

# 注意
教科書で用いられているImputerはもうサポートされていないらしい  
使い方も少し違う  
https://qiita.com/yumarule/items/19c81e16c136676a8573

### SimpleImputerの使い方  

|引数|説明|
|:---:|:---:|
|strategy|欠損値の補完方法 mean , median , most_frequent のいずれかを指定できる|

### カテゴリ変数のエンコーディング
カテゴリ変数とは、数値ではないもの扱っている変数である  
例えば  
テストの選択肢 A,B,Cのいずれかを選んでいる  
みたいな割り当てを行っている変数の事

In [12]:
category = pandas.DataFrame({"A":[1,2,3,4,5,6],
                             "B":["a","a","c","b","e","c"],})
category

,A,B
0,1,a
1,2,a
2,3,c
3,4,b
4,5,e
5,6,c


こういったカテゴリ変数を数値に変換する方法には主に2つある  
* カテゴリ変数のエンコーディング  
* One-hotエンコーディング

### カテゴリ変数のエンコーディング
a -> 0  
b -> 1  
c -> 2  
みたいに数字を割り当てる事

In [13]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

label_encoder.fit(category["B"])

label_encoder.transform(category["B"])

array([0, 0, 2, 1, 3, 2])

In [14]:
# 変換される前の値を確認する
label_encoder.classes_

array(['a', 'b', 'c', 'e'], dtype=object)

### one-hotエンコーディング
カテゴリの種類数分、列を増やし、該当する列に1をそれ以外に0を割り当てるエンコード  
機械学習分野ではよく用いられる方法  
one-hotエンコーディングは**ダミー化**とも呼ばれ、エンコードされた値の事を**ダミー変数**とも呼ぶ  
  
one-hotエンコーディングに関しては、scikit-learn,pandasどちらでも対応できるが、  
pandasの方が見通しが良い

In [25]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

label_encoder = LabelEncoder()
one_hot_encoder = ColumnTransformer([("one-hot-encoding", OneHotEncoder(),
                         [1])], remainder='passthrough')

category_copy = category.copy()

category_copy["B"] = label_encoder.fit_transform(category_copy["B"])
one_hot_encoder.fit_transform(category_copy)

array([[1., 0., 0., 0., 1.],
       [1., 0., 0., 0., 2.],
       [0., 0., 1., 0., 3.],
       [0., 1., 0., 0., 4.],
       [0., 0., 0., 1., 5.],
       [0., 0., 1., 0., 6.]])

# 注意
OneHotEncoderの方も仕様がだいぶ変わっている  
https://network-beginner.xyz/onehotencoder_categorical_features_typeerror

one-hotエンコーディングを行うと、疎行列が生成される  
(多くの成分が0である行列。反対に多くの成分が何かしらの値であるものを密行列という。)  

### 特徴量の正規化
ある一つの行列内に、2ケタの数値と4ケタの数値が混合しているとする  
この時、行列演算を行うと4ケタの数値の影響力が大きすぎて他のデータが軽視されかねない  
  
これを防ぐために、**特徴量の正規化**というものがある  
数値間の大小関係は変えずに、1ケタサイズに数値変換するような処理である  
  
これには主に二つの方法がある  
* 分散正規化(標準化)
* 最小最大正規化


### 分散正規化(標準化)
  
$x' = \frac{x-\mu}{\sigma}$  
$\mu$は平均  
$\sigma$は標準偏差である  
  
それぞれのデータを上記のような式にかけて、変換を行うと、  
平均0,分散1のデータに変換される

In [26]:
data_frame = pandas.DataFrame({"A":[1,2,3,4,5],
                               "b":[100,200,500,800,1000],})
data_frame

,A,b
0,1,100
1,2,200
2,3,500
3,4,800
4,5,1000


In [28]:
from sklearn.preprocessing import StandardScaler

# scikit-learnを用いれば3行で標準化が可能となる
std_scaler = StandardScaler()
std_scaler.fit(data_frame)
std_scaler.transform(data_frame)

array([[-1.41421356, -1.22474487],
       [-0.70710678, -0.93313895],
       [ 0.        , -0.05832118],
       [ 0.70710678,  0.81649658],
       [ 1.41421356,  1.39970842]])

### 最小最大正規化
  
$x' = \frac{x-x_{min}}{x_{max}-x_{min}}$
  
$x_{min}$はデータ中の最小値  
$x_{max}$はデータ中の最大値である  
  
これを行うことで、  
1～0の間で正規化される  

In [29]:
from sklearn.preprocessing import MinMaxScaler

# scikit-learnを用いれば3行で標準化が可能となる
mm_scaler = MinMaxScaler()
mm_scaler.fit(data_frame)
mm_scaler.transform(data_frame)

array([[0.        , 0.        ],
       [0.25      , 0.11111111],
       [0.5       , 0.44444444],
       [0.75      , 0.77777778],
       [1.        , 1.        ]])